In [1]:
from pyspark.sql.functions import col

In [2]:
%run ../src/document_number_parsing_spark

In [3]:
lst = [(603642,'00C074511-23','M2C011965'),
(603643,'no doc','Roger will pick up order tomorrow'),
(603648,'DOCUMENT 00C074555','00C074522'),
(603650,' 03C526338/','<p>ordered antenna for 321&nbsp; 25V1574809</p>'),
(603652,'30C202661&25C158124&25C158888',''),
(603654,'03C526341','sample notes 03C5263414, 03C5263444'),
(603655,'03C526333A','sample notes1'),
(603656,'','00C074535 there are others as well 00C074533')]

In [4]:
df = spark.createDataFrame(sc.parallelize(lst), ['meetingid', 'PartsDocumentNumber', 'Notes'])
df.show()

+---------+--------------------+--------------------+
meetingid| PartsDocumentNumber| Notes|
+---------+--------------------+--------------------+
 603642| 00C074511-23| M2C011965|
 603643| no doc|Roger will pick u...|
 603648| DOCUMENT 00C074555| 00C074522|
 603650| 03C526338/|<p>ordered antenn...|
 603652|30C202661&25C1581...| |
 603654| 03C526341|sample notes 03C5...|
 603655| 03C526333A| sample notes1|
 603656| |00C074535 there a...|
+---------+--------------------+--------------------+

In [5]:
upper_df = convert_to_upper(df)

In [6]:
assert upper_df.filter(col("meetingid") == 603643).select("PartsDocumentNumber").collect()[0][0] == "NO DOC", "PartsDocumentNumber Converted to upper case"
assert upper_df.filter(col("meetingid") == 603655).select("Notes").collect()[0][0] == "SAMPLE NOTES1", "Notes Converted to upper case"

In [7]:
doc_num_extract_df = extract_doc_num_from_pdn(upper_df.drop("Notes"))

In [8]:
assert doc_num_extract_df.filter(col("meetingid") == 603652).select("RootDocumentNumber").count() == 3, "No. of document number is 3"
assert doc_num_extract_df.filter(col("meetingid") == 603642).select("RootDocumentNumber").count() == 2, "No. of document number is 2"
assert doc_num_extract_df.filter(col("meetingid") == 603650).select("RootDocumentNumber").collect()[0][0] == "03C526338", "document number is not parsed correctly"

In [9]:
notes_extract_df = extract_doc_num_from_notes(upper_df.drop("PartsDocumentNumber"))

In [10]:
notes_extract_df.show()

+---------+------------------+
meetingid|RootDocumentNumber|
+---------+------------------+
 603648| 00C074522|
 603656| 00C074535|
 603656| 00C074533|
+---------+------------------+

In [11]:
assert notes_extract_df.filter(col("meetingid") == 603648).select("RootDocumentNumber").collect()[0][0] == "00C074522", "document number is not parsed correctly"
assert notes_extract_df.filter(col("meetingid") == 603654).select("RootDocumentNumber").collect()[0][0] == "03C526341", "document number is not parsed correctly"